## Creating a list of domains pairs we need to compute similarity - only within clans

In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
import pickle

In [2]:
curr_dir = !pwd
pfam_version = "31"
domains_th = "10"

input_path = curr_dir[0]+"/../../2.parse_Pfam/v"+pfam_version+"/"

In [3]:
with open(input_path+"domain_to_clan_dict.pik", 'rb') as handle:
    domain_to_clan_dict = pickle.load(handle)
with open(input_path+"clan_to_domains_dict.pik", 'rb') as handle:
    clan_to_domains_dict = pickle.load(handle)

#### Creating a table of pairs

In [4]:
sim_dom1 = []
sim_dom2 = []
for clan in clan_to_domains_dict.keys():
    clan_size = len(clan_to_domains_dict[clan])
    if (clan_size > 1):
        tmp_clan = clan_to_domains_dict[clan].copy()
        for domain in clan_to_domains_dict[clan]:
            tmp_clan.remove(domain)
            sim_dom1.extend([domain] * (len(tmp_clan)))
            sim_dom2.extend(tmp_clan)

In [5]:
tmp_dict = {"sim_dom1": sim_dom1, "sim_dom2": sim_dom2}
sim_pairs_df = pd.DataFrame(data=tmp_dict)

#### Saving the pairs to a table

In [6]:
sim_pairs_df.to_csv(curr_dir[0]+"/pfam-v"+pfam_version+"/domains_pairs_for_hhalign.csv")

In [7]:
sim_pairs_df.shape

(6851346, 2)

### Create a subset of pairs that have domains with > 10 instances and binding scores

In [8]:
with open(curr_dir[0]+"/../../5.domains_stats/pfam-v"+pfam_version+"/filtered"+domains_th+"_list.pik", 'rb') as handle:
    filtered_domains_list = pickle.load(handle)
filtered_domains_list.sort()

with open(curr_dir[0]+"/../../7.InteracDome_targetVariable/domains_ligands_propensity_dict.pik", 'rb') as handle:
    binding_props_dict = pickle.load(handle)
    
domains_prop_intersection = []
for domain in filtered_domains_list:
    if (domain in binding_props_dict.keys()):
        domains_prop_intersection.append(domain)
print "number of domains = "+str(len(domains_prop_intersection))

number of domains = 443


In [9]:
indices_for_remove = []
for index, pair in sim_pairs_df.iterrows():
    dom1 =  pair["sim_dom1"]
    dom2 =  pair["sim_dom2"]
    if (dom1 not in domains_prop_intersection):
        indices_for_remove.append(index)
    elif (dom2 not in domains_prop_intersection):
        indices_for_remove.append(index)

In [10]:
sim_pairs_df10 = sim_pairs_df.drop(indices_for_remove)
sim_pairs_df10 = sim_pairs_df10.reset_index(drop=True)
sim_pairs_df10.shape

(2764, 2)

### Remove no_clan from the table

In [11]:
drop_idx = []
for index, row in sim_pairs_df10.iterrows():
    clan1 = domain_to_clan_dict[row["sim_dom1"]]
    clan2 = domain_to_clan_dict[row["sim_dom2"]]
    if (clan1 == "No_clan" and clan2 == "No_clan"):
        drop_idx.append(index)

In [12]:
sim_pairs_df10_noclan = sim_pairs_df10.drop(drop_idx)
sim_pairs_df10_noclan = sim_pairs_df10_noclan.reset_index(drop=True)
sim_pairs_df10_noclan.shape

(1279, 2)

In [13]:
sim_pairs_df10.to_csv(curr_dir[0]+"/pfam-v"+pfam_version+"/domains_pairs_for_hhalign_filtered10_noclan.csv")